## Comparing Regression Algorithms on Salaries for Data-Related Jobs

Project Object: Build a model to predict the Salary for Data-related jobs using the provided features/inputs.

Dataset Source: https://www.kaggle.com/datasets/whenamancodes/data-science-fields-salary-categorization

##### Import Necessary Libraries

In [0]:
import pyspark
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler, VectorIndexer
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator

from distutils.version import LooseVersion

from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType
#import pandas as pd

##### Ingest Dataset & Apply Schema

In [0]:
# File location and type
file_location = "/FileStore/tables/data science salaries/regression/Data_Science_Fields_Salary_Categorization.tsv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = "\t"

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

df = df.drop('_c0').collect()

schema = StructType([StructField('Working_Year', StringType(), True), 
                     StructField("Designation", StringType(), True), 
                     StructField("Experience", StringType(), True), 
                     StructField("Employment_Status", StringType(), True), 
                     StructField("Salary_In_Rupees", StringType(), True), 
                     StructField("Employee_Location", StringType(), True),
                     StructField("Company_Location", StringType(), True), 
                     StructField("Company_Size", StringType(), True), 
                     StructField("Remote_work_Ratio", StringType(), True)])

df = spark.createDataFrame(df, schema=schema)

display(df)

Working_Year,Designation,Experience,Employment_Status,Salary_In_Rupees,Employee_Location,Company_Location,Company_Size,Remote_work_Ratio
2020,Data Scientist,MI,FT,"6,352,272.00",DE,DE,L,0
2020,Machine Learning Scientist,SE,FT,"20,688,070.00",JP,JP,S,0
2020,Big Data Engineer,SE,FT,"8,674,985.00",GB,GB,M,50
2020,Product Data Analyst,MI,FT,"1,591,390.00",HN,HN,S,0
2020,Machine Learning Engineer,SE,FT,"11,935,425.00",US,US,L,50
2020,Data Analyst,EN,FT,"5,729,004.00",US,US,L,100
2020,Lead Data Scientist,SE,FT,"15,118,205.00",US,US,S,100
2020,Data Scientist,MI,FT,"2,843,416.00",HU,HU,L,50
2020,Business Data Analyst,MI,FT,"10,741,883.00",US,US,L,100
2020,Lead Data Engineer,SE,FT,"9,946,188.00",NZ,NZ,S,50


In [0]:
df.count()

Out[3]: 607

##### Display Unique Values for Each Feature (& Count of Unique Values for Each Feature)

In [0]:
for col in df:
    print(df.select(col).distinct().count())
    print(df.select(col).distinct().show())
    print('\n\n')

3
+------------+
|Working_Year|
+------------+
|        2020|
|        2021|
|        2022|
+------------+

None



50
+--------------------+
|         Designation|
+--------------------+
|  Lead Data Engineer|
|Machine Learning ...|
|   Lead Data Analyst|
|Data Engineering ...|
|Product Data Analyst|
|        AI Scientist|
|Director of Data ...|
|Machine Learning ...|
| Lead Data Scientist|
|Machine Learning ...|
|Machine Learning ...|
|  Research Scientist|
|      Data Scientist|
|        Data Analyst|
|Data Science Cons...|
|         ML Engineer|
|Business Data Ana...|
|Computer Vision E...|
|   Big Data Engineer|
|     BI Data Analyst|
+--------------------+
only showing top 20 rows

None



4
+----------+
|Experience|
+----------+
|        EX|
|        MI|
|        EN|
|        SE|
+----------+

None



4
+-----------------+
|Employment_Status|
+-----------------+
|               FT|
|               PT|
|               CT|
|               FL|
+-----------------+

None



369
+----

##### Data Preprocessing: Employee_Location (Convert Countries to Continents)

In [0]:
df.select('Employee_Location').distinct().show(65)
df.select('Company_Location').distinct().show(65)

country_to_continent = {'NL' : 'Europe', 'PL' : 'Europe', 'MX' : 'North America', 'CN' : 'Asia', 'AT' : 'Europe', 'RU' : 'Asia', 'HR' : 'Europe', 'PT' : 'Europe', 'PK' : 'Asia', 'CA' : 'North America', 'GB' : 'Europe', 'HN' : 'Asia', 'DE' : 'Europe', 'ES' : 'Europe', 'US' : 'North America', 'IN' : 'Asia', 'AE' : 'Asia', 'FR' : 'Europe', 'GR' : 'Europe', 'IT' : 'Europe', 'PH' : 'Asia', 'JP' : 'Asia', 'NZ' : 'Australia', 'HU' : 'Europe', 'NG' : 'Africa', 'DK' : 'Europe', 'IQ' : 'Asian', 'BR' : 'South America', 'SG' : 'Asia', 'BG' : 'Europe', 'BE' : 'Europe', 'VN' : 'Asia', 'UA' : 'Europe', 'RO' : 'Europe', 'HK' : 'Asia', 'CL' : 'South America', 'MT' : 'Europe', 'MD' : 'Europe', 'IR' : 'Asia', 'KE' : 'Africa', 'SI' : 'Europe', 'CO' : 'South America', 'CZ' : 'Europe', 'TR' : 'Europe', 'RS' : 'Europe', 'JE' : 'Europe', 'PR' : 'North America', 'LU' : 'Europe', 'AR' : 'South America', 'DZ' : 'Africa', 'EE' : 'Europe', 'AU' : 'Australia', 'MY' : 'Asia', 'CH' : 'Europe', 'TN' : 'Africa', 'IE' : 'Europe', 'BO' : 'South America', 'IL': 'Asia', 'AS' : 'Asia'} # I classified 'American Samoa' (AS) as Asian based on Geographical Location.

df = df.replace(country_to_continent,subset=['Employee_Location'])
df.show()

+-----------------+
|Employee_Location|
+-----------------+
|               NL|
|               PL|
|               MX|
|               CN|
|               AT|
|               RU|
|               HR|
|               PT|
|               PK|
|               CA|
|               GB|
|               HN|
|               DE|
|               ES|
|               US|
|               IN|
|               AE|
|               FR|
|               GR|
|               IT|
|               PH|
|               JP|
|               NZ|
|               HU|
|               NG|
|               DK|
|               IQ|
|               BR|
|               SG|
|               BG|
|               BE|
|               VN|
|               UA|
|               RO|
|               HK|
|               CL|
|               MT|
|               MD|
|               IR|
|               KE|
|               SI|
|               CO|
|               CZ|
|               TR|
|               RS|
|               JE|
|               PR|


##### Data Preprocessing: Company_Location (Convert Countries to Continents)

In [0]:
df = df.replace(country_to_continent,subset=['Company_Location'])
df.show()

+------------+--------------------+----------+-----------------+----------------+-----------------+----------------+------------+-----------------+
|Working_Year|         Designation|Experience|Employment_Status|Salary_In_Rupees|Employee_Location|Company_Location|Company_Size|Remote_work_Ratio|
+------------+--------------------+----------+-----------------+----------------+-----------------+----------------+------------+-----------------+
|        2020|      Data Scientist|        MI|               FT|    6,352,272.00|           Europe|          Europe|           L|                0|
|        2020|Machine Learning ...|        SE|               FT|   20,688,070.00|             Asia|            Asia|           S|                0|
|        2020|   Big Data Engineer|        SE|               FT|    8,674,985.00|           Europe|          Europe|           M|               50|
|        2020|Product Data Analyst|        MI|               FT|    1,591,390.00|             Asia|            A

##### Data Preprocessing: Company_Size (Eliminate Shorthand Versions)

In [0]:
comp_size_converter = {'S' : 'Small', 'M' : 'Medium', 'L' : 'Large'}

df = df.replace(comp_size_converter,subset=['Company_Size'])

##### Data Preprocessing: Employment_Status (Eliminate Shorthand Versions)

In [0]:
ee_status_converter = {'FT' : 'Full Time', 'PT' : 'Part Time', 'CT' : 'Contract', 'FL' : 'Freelance'}

df = df.replace(ee_status_converter,subset=['Employment_Status'])

##### Data Preprocessing: Experience (Eliminate Shorthand Versions)

In [0]:
exp_level_converter = {'SE' : 'Senior-Level/Experienced', 'MI' : 'Mid-Level/Intermediate', 'EN' : 'Entry-Level/ Junior', 'EX' : 'Executive-Level'}

df = df.replace(exp_level_converter, subset=['Experience'])

##### Data Preprocessing: Convert Rupees to USD

In [0]:
# Exchange rate as of completing this project is each Ruppee equals 0.012 USD

df = df.withColumn("Salary_In_Rupees", f.regexp_replace('Salary_In_Rupees', ",", "")).alias("Salary_In_Rupees")
df = df.withColumn("Salary_In_USD", round(df.Salary_In_Rupees.cast("double") * 0.012, 2)).alias("label")
df = df.withColumnRenamed("Salary_In_USD", "label")

df = df.drop('Salary_In_Rupees')

##### Check That Changes Worked as Expected

In [0]:
df.printSchema()
df.show()

root
 |-- Working_Year: string (nullable = true)
 |-- Designation: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Employment_Status: string (nullable = true)
 |-- Employee_Location: string (nullable = true)
 |-- Company_Location: string (nullable = true)
 |-- Company_Size: string (nullable = true)
 |-- Remote_work_Ratio: string (nullable = true)
 |-- label: double (nullable = true)

+------------+--------------------+--------------------+-----------------+-----------------+----------------+------------+-----------------+---------+
|Working_Year|         Designation|          Experience|Employment_Status|Employee_Location|Company_Location|Company_Size|Remote_work_Ratio|    label|
+------------+--------------------+--------------------+-----------------+-----------------+----------------+------------+-----------------+---------+
|        2020|      Data Scientist|Mid-Level/Interme...|        Full Time|           Europe|          Europe|       Large|              

##### Visualize Data Relationships

In [0]:
display(df)

Working_Year,Designation,Experience,Employment_Status,Employee_Location,Company_Location,Company_Size,Remote_work_Ratio,label
2020,Data Scientist,Mid-Level/Intermediate,Full Time,Europe,Europe,Large,0,76227.26
2020,Machine Learning Scientist,Senior-Level/Experienced,Full Time,Asia,Asia,Small,0,248256.84
2020,Big Data Engineer,Senior-Level/Experienced,Full Time,Europe,Europe,Medium,50,104099.82
2020,Product Data Analyst,Mid-Level/Intermediate,Full Time,Asia,Asia,Small,0,19096.68
2020,Machine Learning Engineer,Senior-Level/Experienced,Full Time,North America,North America,Large,50,143225.1
2020,Data Analyst,Entry-Level/ Junior,Full Time,North America,North America,Large,100,68748.05
2020,Lead Data Scientist,Senior-Level/Experienced,Full Time,North America,North America,Small,100,181418.46
2020,Data Scientist,Mid-Level/Intermediate,Full Time,Europe,Europe,Large,50,34120.99
2020,Business Data Analyst,Mid-Level/Intermediate,Full Time,North America,North America,Large,100,128902.6
2020,Lead Data Engineer,Senior-Level/Experienced,Full Time,Australia,Australia,Small,50,119354.26


Output can only be rendered in Databricks

Output can only be rendered in Databricks

Output can only be rendered in Databricks

##### Encode Features Appropriately for Model

In [0]:
categorical_columns = ['Working_Year', 'Designation', 'Experience', 'Employment_Status', 'Employee_Location', 'Company_Location', 'Company_Size', 'Remote_work_Ratio']

stages = [] # stages in the pipeline

for cat in categorical_columns:
    #Category Indexing with StringIndexer
    stringIndexer = StringIndexer(inputCol=cat, outputCol=cat + "Index")
    # Use OneHotEncoder to convert categorical variables into binary SparseVectors
    
    if LooseVersion(pyspark.__version__) < LooseVersion("3.0"):
        from pyspark.ml.feature import OneHotEncoderEstimator
        encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[cat + 'classVec'])
    else:
        from pyspark.ml.feature import OneHotEncoder
        encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[cat + 'classVec'])
    # Add the two stages
    stages += [stringIndexer, encoder]

##### Transform All Features into Vector Using VectorAssembler

In [0]:
assembler_inputs = [c + "classVec" for c in categorical_columns]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")
stages += [assembler]

##### Build, Fit, & Transform Pipeline

In [0]:
partial_pipeline = Pipeline().setStages(stages)
pipeline_model = partial_pipeline.fit(df)
prepped_ds = pipeline_model.transform(df)

##### Split Dataset into Train/Test Datasets

In [0]:
train_ds, test_ds = prepped_ds.randomSplit([0.80, 0.20], seed=42)

##### Fit/Train GBTreeRegressor Model

In [0]:
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'label', maxIter=100)
gbt_model = gbt.fit(train_ds)

##### Some Descriptive Statistics for Justification

In [0]:
display(train_ds.describe())

summary,Working_Year,Designation,Experience,Employment_Status,Employee_Location,Company_Location,Company_Size,Remote_work_Ratio,label,Working_YearIndex,DesignationIndex,ExperienceIndex,Employment_StatusIndex,Employee_LocationIndex,Company_LocationIndex,Company_SizeIndex,Remote_work_RatioIndex
count,476,476,476,476,476,476,476,476,476,476,476,476,476,476,476,476,476
mean,2021.40756302521,null,null,null,null,null,null,71.95378151260505,107706.97737394957,0.592436974789916,6.199579831932773,0.7815126050420168,0.05042016806722689,0.6092436974789915,0.5147058823529411,0.5840336134453782,0.5042016806722689
stddev,0.691196937598629,null,null,null,null,null,null,40.602903271701024,69178.40084220623,0.6911969375986675,9.741721442598964,0.8624046075997042,0.3333716622515968,0.8936257638126087,0.8193669685203128,0.716186766970423,0.7412529884752473
min,2020,AI Scientist,Entry-Level/ Junior,Contract,Africa,Africa,Large,0,2729.87,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,2022,Staff Data Scientist,Senior-Level/Experienced,Part Time,South America,South America,Small,50,572900.4,2.0,49.0,3.0,3.0,6.0,6.0,2.0,2.0


##### Make Predictions Using Testing Dataset

In [0]:
gbt_preds = gbt_model.transform(test_ds)
gbt_preds.select('prediction', 'label', 'features').show(5)

+------------------+---------+--------------------+
|        prediction|    label|            features|
+------------------+---------+--------------------+
|150949.74168933113| 66838.38|(73,[9,53,54,57,6...|
|108357.94794295909| 128902.6|(73,[21,52,54,57,...|
|48180.578797897106|  9548.34|(73,[4,53,54,61,6...|
| 44502.88253379664| 44647.08|(73,[4,52,54,58,6...|
| 58911.67959541284|107774.03|(73,[3,52,54,58,6...|
+------------------+---------+--------------------+
only showing top 5 rows



##### Evaluate Predictions

In [0]:
gbt_evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

rmse = gbt_evaluator.evaluate(gbt_preds)

print(f"Root Mean Squared Error (RMSE) on test data is ${rmse}")

Root Mean Squared Error (RMSE) on test data is $57072.99958417747


##### Save Model

In [0]:
gbt_model.write().overwrite().save("Data Salary Predictor - Regression Model")

##### How I Would Reload Model (If Needed)

## Notes & Other Takeaways From This Project
****
- The results were not what I was hoping for, but here are the reasons why this project did not work out well:

  - In Compensation Analysis, the factors include factors such as: experience, education, responsibility, job complexity, supervision received, supervision exercised, consequences of error, working conditions, etc. The goal is to provide objective factors that evaluate the value of the job. The factors provided here are helpful, but are most likely do not have the largest impact on salary determination.
  - The closest that this analysis gets to those is with a categorical feature representing experience. It breaks a full career into four (4) different groups. That removes information that could be used to more accurately determine salary. 
  - As alluded to in the previous point, the inputs are (with the potential exception of the 'Remote Work Ratio' feature) all categorical/discrete. For a regression analysis like this, multiple continuous/numerical features would have improved the results immensely. Some of the values can be categorical/discrete in nature.
  - The dataset size is too small, especially for the type of data. This dataset includes just over six hundred (600) samples. Combine that with all (or nearly all) discrete features and it is likely to cause issues.
  - In my experience, there tends to be some crossover or confusion in how different companies classify data jobs. Every company determines what is the difference between a Data Analyst vs Data Scientist differently, so the job title may have less importance. Additionally, when there are approximately fifty (50) different job titles in an analysis of approximately six hundred (600) samples, there are not enough of each job title for there to provide any significance to the job title feature.
****
- In order to cure the poor results is to collect more data that includes:
  
  - More samples
  - More continuous values
  - Features that better evaluate the value of different jobs
****
- Next improvements I am looking to incorporate into Apache Spark-related projects is wrapping my work in functions.
****